In [2]:
import tensorflow as tf
!wget https://s3.amazonaws.com/keras-datasets/ames-housing-dataset.zip

--2019-06-03 14:15:26--  https://s3.amazonaws.com/keras-datasets/ames-housing-dataset.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.110.205
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.110.205|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 182871 (179K) [application/zip]
Saving to: ‘ames-housing-dataset.zip’

ames-housing-datase 100%[===================>] 178.58K   776KB/s    in 0.2s    

2019-06-03 14:15:27 (776 KB/s) - ‘ames-housing-dataset.zip’ saved [182871/182871]



In [3]:
!unzip ames-housing-dataset.zip

Archive:  ames-housing-dataset.zip
  inflating: AmesHousing.csv         


In [9]:
!cat AmesHousing.csv

cat: AmesHousing.csv: Permission denied


In [11]:
import pandas as pd
import numpy as np

df = pd.read_csv("AmesHousing.csv")
df.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [13]:
print('Columns:', list(enumerate(df.columns)))

Columns: [(0, 'Order'), (1, 'PID'), (2, 'MS SubClass'), (3, 'MS Zoning'), (4, 'Lot Frontage'), (5, 'Lot Area'), (6, 'Street'), (7, 'Alley'), (8, 'Lot Shape'), (9, 'Land Contour'), (10, 'Utilities'), (11, 'Lot Config'), (12, 'Land Slope'), (13, 'Neighborhood'), (14, 'Condition 1'), (15, 'Condition 2'), (16, 'Bldg Type'), (17, 'House Style'), (18, 'Overall Qual'), (19, 'Overall Cond'), (20, 'Year Built'), (21, 'Year Remod/Add'), (22, 'Roof Style'), (23, 'Roof Matl'), (24, 'Exterior 1st'), (25, 'Exterior 2nd'), (26, 'Mas Vnr Type'), (27, 'Mas Vnr Area'), (28, 'Exter Qual'), (29, 'Exter Cond'), (30, 'Foundation'), (31, 'Bsmt Qual'), (32, 'Bsmt Cond'), (33, 'Bsmt Exposure'), (34, 'BsmtFin Type 1'), (35, 'BsmtFin SF 1'), (36, 'BsmtFin Type 2'), (37, 'BsmtFin SF 2'), (38, 'Bsmt Unf SF'), (39, 'Total Bsmt SF'), (40, 'Heating'), (41, 'Heating QC'), (42, 'Central Air'), (43, 'Electrical'), (44, '1st Flr SF'), (45, '2nd Flr SF'), (46, 'Low Qual Fin SF'), (47, 'Gr Liv Area'), (48, 'Bsmt Full Bath'

In [14]:
targets = df['SalePrice'].values.astype('float32')
print('targets.shape:', targets.shape)

targets.shape: (2930,)


In [15]:
#is this different than nonvectorized? We still have to onehot encode category
#
def vectorize_column(column):
    print('Processing column: "%s"' % column.name)
    dtype = column.dtype
    if np.issubdtype(dtype, np.number):
        print('...numerical column')
        array = vectorize_numerical_column(column)
    else:
        print('...categorical column')
        array = vectorize_categorical_column(column)
    print('...shape:', array.shape)
    return array

  
def vectorize_numerical_column(column):
    return np.expand_dims(column.values, -1).astype('float32')


def vectorize_categorical_column(column):
    print('...value counts:')
    print(column.value_counts())
  
  # Filter values that occur at least SKIP_VALUES_WITH_FEWER_OCCURENCES_THAN
  # and assign an integer index to each of these values (for one-hot encoding).
    value_index = {}
    for value, count in column.value_counts().items():
        if count >= SKIP_VALUES_WITH_FEWER_OCCURENCES_THAN:
            value_index[value] = len(value_index)
  # Note that our `value_index` 
    print('...kept values:', list(value_index.keys()))
  
  # One-hot encode the filtered values.
    array = np.zeros(shape=(len(column), len(value_index)), dtype='float32')
    for i, value in enumerate(column):
        if value in value_index:
            index = value_index[value]
            array[i, index] = 1.
    return array

In [19]:
SKIP_VALUES_WITH_FEWER_OCCURENCES_THAN = 50

column_arrays = []
# We exclude column 0 and 1 (not informative) and the last column (targets)
for name in df.columns[2:-1]:
    array = vectorize_column(df[name])
    column_arrays.append(array)
features = np.concatenate(column_arrays, axis=-1)
print('features shape: %s' % (features.shape,))

Processing column: "MS SubClass"
...numerical column
...shape: (2930, 1)
Processing column: "MS Zoning"
...categorical column
...value counts:
RL         2273
RM          462
FV          139
RH           27
C (all)      25
I (all)       2
A (agr)       2
Name: MS Zoning, dtype: int64
...kept values: ['RL', 'RM', 'FV']
...shape: (2930, 3)
Processing column: "Lot Frontage"
...numerical column
...shape: (2930, 1)
Processing column: "Lot Area"
...numerical column
...shape: (2930, 1)
Processing column: "Street"
...categorical column
...value counts:
Pave    2918
Grvl      12
Name: Street, dtype: int64
...kept values: ['Pave']
...shape: (2930, 1)
Processing column: "Alley"
...categorical column
...value counts:
Grvl    120
Pave     78
Name: Alley, dtype: int64
...kept values: ['Grvl', 'Pave']
...shape: (2930, 2)
Processing column: "Lot Shape"
...categorical column
...value counts:
Reg    1859
IR1     979
IR2      76
IR3      16
Name: Lot Shape, dtype: int64
...kept values: ['Reg', 'IR1', 'IR

...shape: (2930, 1)
Processing column: "Half Bath"
...numerical column
...shape: (2930, 1)
Processing column: "Bedroom AbvGr"
...numerical column
...shape: (2930, 1)
Processing column: "Kitchen AbvGr"
...numerical column
...shape: (2930, 1)
Processing column: "Kitchen Qual"
...categorical column
...value counts:
TA    1494
Gd    1160
Ex     205
Fa      70
Po       1
Name: Kitchen Qual, dtype: int64
...kept values: ['TA', 'Gd', 'Ex', 'Fa']
...shape: (2930, 4)
Processing column: "TotRms AbvGrd"
...numerical column
...shape: (2930, 1)
Processing column: "Functional"
...categorical column
...value counts:
Typ     2728
Min2      70
Min1      65
Mod       35
Maj1      19
Maj2       9
Sal        2
Sev        2
Name: Functional, dtype: int64
...kept values: ['Typ', 'Min2', 'Min1']
...shape: (2930, 3)
Processing column: "Fireplaces"
...numerical column
...shape: (2930, 1)
Processing column: "Fireplace Qu"
...categorical column
...value counts:
Gd    744
TA    600
Fa     75
Po     46
Ex     43
N

In [20]:
column_arrays = []
# We exclude column 0 and 1 (not informative) and the last column (targets)
for name in df.columns[2:-1]:
    array = vectorize_column(df[name])
    column_arrays.append(array)
features = np.concatenate(column_arrays, axis=-1)
print('features shape: %s' % (features.shape,))

Processing column: "MS SubClass"
...numerical column
...shape: (2930, 1)
Processing column: "MS Zoning"
...categorical column
...value counts:
RL         2273
RM          462
FV          139
RH           27
C (all)      25
I (all)       2
A (agr)       2
Name: MS Zoning, dtype: int64
...kept values: ['RL', 'RM', 'FV']
...shape: (2930, 3)
Processing column: "Lot Frontage"
...numerical column
...shape: (2930, 1)
Processing column: "Lot Area"
...numerical column
...shape: (2930, 1)
Processing column: "Street"
...categorical column
...value counts:
Pave    2918
Grvl      12
Name: Street, dtype: int64
...kept values: ['Pave']
...shape: (2930, 1)
Processing column: "Alley"
...categorical column
...value counts:
Grvl    120
Pave     78
Name: Alley, dtype: int64
...kept values: ['Grvl', 'Pave']
...shape: (2930, 2)
Processing column: "Lot Shape"
...categorical column
...value counts:
Reg    1859
IR1     979
IR2      76
IR3      16
Name: Lot Shape, dtype: int64
...kept values: ['Reg', 'IR1', 'IR

...shape: (2930, 2)
Processing column: "Electrical"
...categorical column
...value counts:
SBrkr    2682
FuseA     188
FuseF      50
FuseP       8
Mix         1
Name: Electrical, dtype: int64
...kept values: ['SBrkr', 'FuseA', 'FuseF']
...shape: (2930, 3)
Processing column: "1st Flr SF"
...numerical column
...shape: (2930, 1)
Processing column: "2nd Flr SF"
...numerical column
...shape: (2930, 1)
Processing column: "Low Qual Fin SF"
...numerical column
...shape: (2930, 1)
Processing column: "Gr Liv Area"
...numerical column
...shape: (2930, 1)
Processing column: "Bsmt Full Bath"
...numerical column
...shape: (2930, 1)
Processing column: "Bsmt Half Bath"
...numerical column
...shape: (2930, 1)
Processing column: "Full Bath"
...numerical column
...shape: (2930, 1)
Processing column: "Half Bath"
...numerical column
...shape: (2930, 1)
Processing column: "Bedroom AbvGr"
...numerical column
...shape: (2930, 1)
Processing column: "Kitchen AbvGr"
...numerical column
...shape: (2930, 1)
Proces

In [21]:
num_val_samples = int(len(features) * 0.2)

train_features = features[:-num_val_samples]
train_targets = targets[:-num_val_samples]

val_features = features[-num_val_samples:]
val_targets = targets[-num_val_samples:]

In [22]:
print('Number of training samples:', len(train_features))
print('Number of validation samples:', len(val_features))

Number of training samples: 2344
Number of validation samples: 586


In [23]:
nan_indices = np.where(np.isnan(train_features))
print('Number of NaN entries in train_features:', len(np.unique(nan_indices)))

Number of NaN entries in train_features: 547


In [24]:
features_mean = np.nanmean(train_features, axis=0)

nan_indices = np.where(np.isnan(train_features))
train_features[nan_indices] = np.take(features_mean, nan_indices[1])

nan_indices = np.where(np.isnan(val_features))
val_features[nan_indices] = np.take(features_mean, nan_indices[1])

In [25]:
train_features -= features_mean
val_features -= features_mean

features_std = np.std(train_features, axis=0)
train_features /= features_std
val_features /= features_std

In [26]:
targets_median = np.median(train_targets, axis=0)
train_targets /= targets_median
val_targets /= targets_median

In [27]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=features.shape[1:])
x = layers.Dense(128, activation='relu')(inputs)
x = layers.Dropout(0.5)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)
deep_output = layers.Dense(1)(x)

wide_output = layers.Dense(1)(inputs)
total_output = layers.add([deep_output, wide_output])

model = keras.Model(inputs, total_output, name='wide_and_deep')
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 183)          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 128)          23552       input_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 128)          0           dense[0][0]                      
__________________________________________________________________________________________________
dense_1 (Dense)      

In [28]:
model.compile(optimizer='adam',
              loss='mse',
              metrics=['mean_absolute_percentage_error'])

callbacks = [keras.callbacks.ModelCheckpoint(
    'ames_regression_model_at_epoch_{epoch}.h5')]

model.fit(train_features, train_targets,
          batch_size=512,
          epochs=500,
          verbose=2,
          callbacks=callbacks,
          validation_data=(val_features, val_targets))

Instructions for updating:
Use tf.cast instead.
Train on 2344 samples, validate on 586 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/500
 - 1s - loss: 5.4649 - mean_absolute_percentage_error: 187.8287 - val_loss: 2.2926 - val_mean_absolute_percentage_error: 136.3727
Epoch 2/500
 - 0s - loss: 3.3771 - mean_absolute_percentage_error: 153.2865 - val_loss: 1.7688 - val_mean_absolute_percentage_error: 116.2364
Epoch 3/500
 - 0s - loss: 2.5902 - mean_absolute_percentage_error: 133.1248 - val_loss: 1.2198 - val_mean_absolute_percentage_error: 93.7424
Epoch 4/500
 - 0s - loss: 2.0538 - mean_absolute_percentage_error: 117.9061 - val_loss: 1.1439 - val_mean_absolute_percentage_error: 90.4467
Epoch 5/500
 - 0s - loss: 1.9010 - mean_absolute_percentage_error: 114.2458 - val_loss: 1.0603 - val_mean_absolute_percentage_error: 87.1898
Epoch 6/500
 - 0s - loss: 1.6313 - mean_absolute_percentage_error: 105.1187 - val_loss: 0.9496 - val_mean_absolute_percentage_error: 82.5330
Epoch 7/5

Epoch 55/500
 - 0s - loss: 0.1533 - mean_absolute_percentage_error: 31.5837 - val_loss: 0.3200 - val_mean_absolute_percentage_error: 50.9241
Epoch 56/500
 - 0s - loss: 0.1618 - mean_absolute_percentage_error: 31.7944 - val_loss: 0.3125 - val_mean_absolute_percentage_error: 49.9291
Epoch 57/500
 - 0s - loss: 0.1550 - mean_absolute_percentage_error: 31.4530 - val_loss: 0.3159 - val_mean_absolute_percentage_error: 50.3679
Epoch 58/500
 - 0s - loss: 0.1658 - mean_absolute_percentage_error: 32.3913 - val_loss: 0.3134 - val_mean_absolute_percentage_error: 50.3866
Epoch 59/500
 - 0s - loss: 0.1486 - mean_absolute_percentage_error: 31.5541 - val_loss: 0.2999 - val_mean_absolute_percentage_error: 49.1972
Epoch 60/500
 - 0s - loss: 0.1557 - mean_absolute_percentage_error: 30.6498 - val_loss: 0.2890 - val_mean_absolute_percentage_error: 48.1753
Epoch 61/500
 - 0s - loss: 0.1479 - mean_absolute_percentage_error: 30.2073 - val_loss: 0.2975 - val_mean_absolute_percentage_error: 48.9785
Epoch 62/500


 - 0s - loss: 0.0745 - mean_absolute_percentage_error: 20.8782 - val_loss: 0.1419 - val_mean_absolute_percentage_error: 33.1006
Epoch 114/500
 - 0s - loss: 0.0722 - mean_absolute_percentage_error: 20.8107 - val_loss: 0.1325 - val_mean_absolute_percentage_error: 31.6459
Epoch 115/500
 - 0s - loss: 0.0749 - mean_absolute_percentage_error: 21.1188 - val_loss: 0.1337 - val_mean_absolute_percentage_error: 32.2442
Epoch 116/500
 - 0s - loss: 0.0710 - mean_absolute_percentage_error: 20.9204 - val_loss: 0.1370 - val_mean_absolute_percentage_error: 32.8141
Epoch 117/500
 - 0s - loss: 0.0724 - mean_absolute_percentage_error: 20.4636 - val_loss: 0.1352 - val_mean_absolute_percentage_error: 32.7327
Epoch 118/500
 - 0s - loss: 0.0674 - mean_absolute_percentage_error: 20.1909 - val_loss: 0.1298 - val_mean_absolute_percentage_error: 31.9068
Epoch 119/500
 - 0s - loss: 0.0677 - mean_absolute_percentage_error: 19.9611 - val_loss: 0.1248 - val_mean_absolute_percentage_error: 31.1492
Epoch 120/500
 - 0s 

Epoch 171/500
 - 0s - loss: 0.0453 - mean_absolute_percentage_error: 15.5816 - val_loss: 0.0746 - val_mean_absolute_percentage_error: 22.5138
Epoch 172/500
 - 0s - loss: 0.0474 - mean_absolute_percentage_error: 15.9270 - val_loss: 0.0692 - val_mean_absolute_percentage_error: 21.3458
Epoch 173/500
 - 0s - loss: 0.0448 - mean_absolute_percentage_error: 15.6026 - val_loss: 0.0661 - val_mean_absolute_percentage_error: 20.9240
Epoch 174/500
 - 0s - loss: 0.0440 - mean_absolute_percentage_error: 15.4657 - val_loss: 0.0671 - val_mean_absolute_percentage_error: 21.2150
Epoch 175/500
 - 0s - loss: 0.0439 - mean_absolute_percentage_error: 15.7623 - val_loss: 0.0684 - val_mean_absolute_percentage_error: 21.3989
Epoch 176/500
 - 0s - loss: 0.0459 - mean_absolute_percentage_error: 15.6217 - val_loss: 0.0712 - val_mean_absolute_percentage_error: 21.8770
Epoch 177/500
 - 0s - loss: 0.0418 - mean_absolute_percentage_error: 14.8791 - val_loss: 0.0711 - val_mean_absolute_percentage_error: 21.7150
Epoch 

Epoch 229/500
 - 0s - loss: 0.0342 - mean_absolute_percentage_error: 12.8978 - val_loss: 0.0453 - val_mean_absolute_percentage_error: 15.8352
Epoch 230/500
 - 0s - loss: 0.0341 - mean_absolute_percentage_error: 13.1688 - val_loss: 0.0443 - val_mean_absolute_percentage_error: 15.4194
Epoch 231/500
 - 0s - loss: 0.0314 - mean_absolute_percentage_error: 12.5806 - val_loss: 0.0430 - val_mean_absolute_percentage_error: 15.4195
Epoch 232/500
 - 0s - loss: 0.0337 - mean_absolute_percentage_error: 13.3183 - val_loss: 0.0421 - val_mean_absolute_percentage_error: 15.5034
Epoch 233/500
 - 0s - loss: 0.0343 - mean_absolute_percentage_error: 13.0028 - val_loss: 0.0428 - val_mean_absolute_percentage_error: 15.5692
Epoch 234/500
 - 0s - loss: 0.0309 - mean_absolute_percentage_error: 12.4740 - val_loss: 0.0438 - val_mean_absolute_percentage_error: 15.4436
Epoch 235/500
 - 0s - loss: 0.0335 - mean_absolute_percentage_error: 12.8861 - val_loss: 0.0441 - val_mean_absolute_percentage_error: 15.4720
Epoch 

Epoch 287/500
 - 0s - loss: 0.0278 - mean_absolute_percentage_error: 11.2135 - val_loss: 0.0319 - val_mean_absolute_percentage_error: 12.4674
Epoch 288/500
 - 0s - loss: 0.0283 - mean_absolute_percentage_error: 11.3930 - val_loss: 0.0332 - val_mean_absolute_percentage_error: 12.6084
Epoch 289/500
 - 0s - loss: 0.0286 - mean_absolute_percentage_error: 11.2853 - val_loss: 0.0336 - val_mean_absolute_percentage_error: 13.1912
Epoch 290/500
 - 0s - loss: 0.0278 - mean_absolute_percentage_error: 11.3851 - val_loss: 0.0335 - val_mean_absolute_percentage_error: 12.9764
Epoch 291/500
 - 0s - loss: 0.0276 - mean_absolute_percentage_error: 11.5476 - val_loss: 0.0335 - val_mean_absolute_percentage_error: 12.9917
Epoch 292/500
 - 0s - loss: 0.0282 - mean_absolute_percentage_error: 11.1634 - val_loss: 0.0343 - val_mean_absolute_percentage_error: 12.8298
Epoch 293/500
 - 0s - loss: 0.0269 - mean_absolute_percentage_error: 11.0459 - val_loss: 0.0345 - val_mean_absolute_percentage_error: 12.8429
Epoch 

Epoch 345/500
 - 0s - loss: 0.0234 - mean_absolute_percentage_error: 10.1396 - val_loss: 0.0308 - val_mean_absolute_percentage_error: 11.4668
Epoch 346/500
 - 0s - loss: 0.0233 - mean_absolute_percentage_error: 10.1499 - val_loss: 0.0318 - val_mean_absolute_percentage_error: 11.9106
Epoch 347/500
 - 0s - loss: 0.0233 - mean_absolute_percentage_error: 10.0999 - val_loss: 0.0311 - val_mean_absolute_percentage_error: 11.5178
Epoch 348/500
 - 0s - loss: 0.0237 - mean_absolute_percentage_error: 10.2316 - val_loss: 0.0304 - val_mean_absolute_percentage_error: 11.2553
Epoch 349/500
 - 0s - loss: 0.0245 - mean_absolute_percentage_error: 10.1643 - val_loss: 0.0294 - val_mean_absolute_percentage_error: 11.3690
Epoch 350/500
 - 0s - loss: 0.0242 - mean_absolute_percentage_error: 10.2119 - val_loss: 0.0285 - val_mean_absolute_percentage_error: 11.2567
Epoch 351/500
 - 0s - loss: 0.0227 - mean_absolute_percentage_error: 10.1414 - val_loss: 0.0285 - val_mean_absolute_percentage_error: 11.2456
Epoch 

Epoch 403/500
 - 0s - loss: 0.0211 - mean_absolute_percentage_error: 9.7559 - val_loss: 0.0271 - val_mean_absolute_percentage_error: 10.9936
Epoch 404/500
 - 0s - loss: 0.0212 - mean_absolute_percentage_error: 9.5559 - val_loss: 0.0265 - val_mean_absolute_percentage_error: 10.5844
Epoch 405/500
 - 0s - loss: 0.0219 - mean_absolute_percentage_error: 9.5266 - val_loss: 0.0267 - val_mean_absolute_percentage_error: 10.3098
Epoch 406/500
 - 0s - loss: 0.0215 - mean_absolute_percentage_error: 9.6690 - val_loss: 0.0274 - val_mean_absolute_percentage_error: 10.8413
Epoch 407/500
 - 0s - loss: 0.0221 - mean_absolute_percentage_error: 9.5532 - val_loss: 0.0262 - val_mean_absolute_percentage_error: 10.8084
Epoch 408/500
 - 0s - loss: 0.0202 - mean_absolute_percentage_error: 9.5079 - val_loss: 0.0259 - val_mean_absolute_percentage_error: 10.3653
Epoch 409/500
 - 0s - loss: 0.0210 - mean_absolute_percentage_error: 9.4463 - val_loss: 0.0264 - val_mean_absolute_percentage_error: 10.4980
Epoch 410/500

Epoch 462/500
 - 0s - loss: 0.0199 - mean_absolute_percentage_error: 9.2582 - val_loss: 0.0266 - val_mean_absolute_percentage_error: 10.5402
Epoch 463/500
 - 0s - loss: 0.0190 - mean_absolute_percentage_error: 8.9500 - val_loss: 0.0257 - val_mean_absolute_percentage_error: 10.3672
Epoch 464/500
 - 0s - loss: 0.0197 - mean_absolute_percentage_error: 9.0616 - val_loss: 0.0252 - val_mean_absolute_percentage_error: 10.2472
Epoch 465/500
 - 0s - loss: 0.0215 - mean_absolute_percentage_error: 9.4470 - val_loss: 0.0257 - val_mean_absolute_percentage_error: 10.7593
Epoch 466/500
 - 0s - loss: 0.0201 - mean_absolute_percentage_error: 9.0689 - val_loss: 0.0262 - val_mean_absolute_percentage_error: 10.6548
Epoch 467/500
 - 0s - loss: 0.0209 - mean_absolute_percentage_error: 9.1819 - val_loss: 0.0258 - val_mean_absolute_percentage_error: 10.3734
Epoch 468/500
 - 0s - loss: 0.0190 - mean_absolute_percentage_error: 9.2902 - val_loss: 0.0245 - val_mean_absolute_percentage_error: 9.8888
Epoch 469/500


In [29]:
preds = model.predict(val_features[:10])
preds *= targets_median
preds = [int(p) for p in preds]
print('Price predictions for first 10 samples:', preds)
print('Actual prices:', [int(p) for p in val_targets[:10] * targets_median])

Price predictions for first 10 samples: [224977, 170545, 151645, 214665, 178002, 193922, 206070, 165336, 179353, 254506]
Actual prices: [235000, 190000, 169000, 241500, 188899, 207500, 250000, 179900, 172000, 294323]


In [ ]:
55